# The passport problem

This notebook illustrates basic data manipulation in Julia and formulation of a set covering problem in JuMP.

In [ ]:
using CSV
using DataFrames
using JuMP
using HiGHS

In [ ]:
const DATA_DIR = joinpath(@__DIR__, "data")

The [Passport Index Dataset](https://github.com/ilyankou/passport-index-dataset) lists travel visa requirements for 199 countries, in .csv format. The version in this repository is updated as of July 3, 2022.

In [ ]:
passport_data = CSV.read(
    joinpath(DATA_DIR, "passport-index-matrix.csv"),
    DataFrames.DataFrame,
)

The first column represents the passport held. The remaining columns indicate the destination.
What countries are included in the list?

In [ ]:
all_countries = passport_data[!, "Passport"]

Let's transform this into a 199 x 199 matrix.

In [ ]:
raw_passport_matrix = Matrix(passport_data[:, all_countries])

Let's explore the values in the dataset.

In [ ]:
raw_passport_matrix[7, 4] # Visa requirements for Argentina passport holders entering Andorra

This means that Argentina passport holders can stay in Andorra for 90 days without a visa.

In [ ]:
print(unique(raw_passport_matrix)) # All distinct values in the matrix.

In [ ]:
# The following function takes a code from the matrix and returns true
# when entry is allowed without a visa or when a visa is granted on arrival,
# and false otherwise.
function can_we_enter(code)
    if code == "visa free" || code == "visa on arrival" || code isa Int
        return true
    end
    try
        val = parse(Int, code)
        return true
    catch
        return false
    end
end

In [ ]:
can_we_enter("visa required")

In [ ]:
can_we_enter(90)

In [ ]:
can_we_enter("90")

In [ ]:
can_we_enter("e-visa")

Now, create a boolean matrix where entry (i,j) indicates if passport holders from country i can enter country j without a visa (or visa on arrival).

In [ ]:
passport_coverage = map(can_we_enter, raw_passport_matrix)

Finally, we can pose the optimization problem:

**What's the smallest number of passports needed to enable visa-free travel to all countries?**

In [ ]:
# Create a new JuMP model using the HiGHS solver.
model = Model(HiGHS.Optimizer)

In [ ]:
# Define a vector of binary decision variables, one for each passport that can be held.
@variable(model, passport_taken[i=1:length(all_countries)], Bin);

In [ ]:
# The objective is to minimize the number of passports held.
@objective(model, Min, sum(passport_taken));

**Exercise**: Add constraints to ensure that all countries are covered.

In [ ]:
@constraint(model, #= FILL IN HERE =# .>= 1);
# Alternatively, you could write a loop:
# for i in 1:length(all_countries)
#   @constraint(model, #= FILL IN HERE =# >= 1)
# end

In [ ]:
optimize!(model)

In [ ]:
termination_status(model)

In [ ]:
println("Minimum number of passports needed: ", objective_value(model))

In [ ]:
print(all_countries[value.(passport_taken) .>= 0.99])

**Exercise:** Which countries *must* be included because they require visas from all others?

As a sanity check, are they included in your optimal solution?

*Hint:* ``sum(passport_coverage, dims=1)`` sums along the rows of the ``passport_coverage`` matrix

In [ ]:
all_countries[#= FILL IN HERE =#]
# Alternatively, write a for loop.